# Data reduction workflow for files generated on Larmor instrument

Some addhoc approximations will need to be done in order to satisfy SANS2D workflow, e.g direct measurments, tramsmission fraction. 

In [1]:
import matplotlib.pyplot as plt
import scipp as sc
from ess import loki, sans
import scippneutron as scn

## Define reduction workflow parameters

We define here whether to include the effects of gravity,
as well as common time-of-flight, wavelength and $Q$ bins for all the measurements.

We also define a range of wavelengths for the monitors that are considered to not be part of the background.

In [2]:
# Include effects of gravity?
gravity = True

#TODO: Check whether this one is needed.
tof_bins = sc.linspace(dim='tof', start=0, stop=100000, num=2, unit='us')

wavelength_bins = sc.linspace(dim='wavelength', start=0.9, stop=13.5, num=110,
                              unit='angstrom')

q_bins = sc.linspace(dim='Q', start=0.008, stop=0.6, num=55, unit='1/angstrom')

# Define the range of wavelengths for the monitors that are considered
# to not be part of the background
monitor_non_background_range = sc.array(dims=['wavelength'],
                                        values=[0.7, 17.1], unit='angstrom')

## Loading data files

We load the following files:

- The direct beam function for the main detector (gives detector efficiency as a function of wavelength)
- The sample measurement
- The direct measurement: this is the run with the empty sample holder/cuvette
- the background measurement: this is the run with only the solvent which the sample is placed in

In [3]:
ds = sc.Dataset()

sample_run_number = 49338
sample_transmission_run_number = 49339
background_run_number = 49334
background_transmission_run_number = 49335
path = 'Larmor_data'
direct_beam = loki.io.load_rkh_wav(f'{path}/DirectBeam_20feb_full_v3.dat')

ds['sample'] = scn.load(filename=f'{path}/LARMOR000{sample_run_number}.nxs')

ds['sample_transmission'] = scn.load(filename=f'{path}/LARMOR000{sample_transmission_run_number}.nxs')

#TODO: How about direct?
ds['direct'] = scn.load(filename=f'{path}/LARMOR000{background_run_number}.nxs')

ds['background'] = scn.load(filename=f'{path}/LARMOR000{background_run_number}.nxs')

ds['background_transmission'] = scn.load(filename=f'{path}/LARMOR000{background_transmission_run_number}.nxs')

ds

FrameworkManager-[Notice] Welcome to Mantid 6.0.20210412.848
FrameworkManager-[Notice] Please cite: http://dx.doi.org/10.1016/j.nima.2014.07.029 and this release: http://dx.doi.org/10.5286/Software/Mantid
CheckMantidVersion-[Notice] A new version of Mantid(6.3.0) is available for download from https://download.mantidproject.org
DownloadInstrument-[Notice] All instrument definitions up to date
LoadRKH-[Notice] LoadRKH started
LoadRKH-[Notice] LoadRKH successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
Load-[Notice] Load started
Load-[Notice] Load successful, Duration 12.93 seconds
ExtractMonitors-[Notice] ExtractMonitors started
ExtractMonitors-[Notice] ExtractMonitors successful, Duration 0.20 seconds


Workspace run log 'good_frames' has unrecognised units: 'frames'
Workspace run log 'raw_frames' has unrecognised units: 'frames'


ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] 

Workspace run log 'good_frames' has unrecognised units: 'frames'
Workspace run log 'raw_frames' has unrecognised units: 'frames'


ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] 

Workspace run log 'good_frames' has unrecognised units: 'frames'
Workspace run log 'raw_frames' has unrecognised units: 'frames'


DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds


DimensionError: Inconsistent size for dim 'spectrum', given 114688, requested 10

## Apply corrections to pixel positions

We apply some corrections to the detector pixel and monitor positions,
as the geometry stored in the file is inaccurate.

In [ ]:
sample_pos_z_offset = 0.3053 * sc.units.m
bench_pos_y_offset = 0.001 * sc.units.m

ds.coords["pixel_width"] = 0.0075 * sc.units.m
ds.coords["pixel_height"] = 0.0117188 * sc.units.m

# Change sample position
ds.coords["sample_position"].fields.z += sample_pos_z_offset
# Apply bench offset to pixel positions
ds.coords["position"].fields.y += bench_pos_y_offset

#TOOD: We don't have this one either
#for key in ds:
#    ds[key].attrs["monitor4"].value.coords["position"].fields.z += monitor4_pos_z_offset

## Masking

The next step is to mask noisy and saturated pixels,
as well as a time-of-flight range that contains spurious artifacts from the beamline components.

**Note:** We use programatic masks here and not those stored in xml files.

### Mask bad pixels

We mask the edges of the detector, which are usually noisy.
We also mask the region close to the center of the beam,
so as to not include saturated pixels in our data reduction.

In [ ]:
x = ds.coords['position'].fields.x
y = ds.coords['position'].fields.y
mask_center = sc.less(sc.sqrt(x*x+y*y), 0.045 * sc.units.m)
mask_edges = sc.greater(sc.abs(x), 0.36 * sc.units.m) # roughly all det IDs listed in original
#MaskDetectorsInShape(Workspace=maskWs, ShapeXML=self.maskingPlaneXML) # irrelevant tiny wedge?

for key in ds:
    ds[key].masks['edges'] = mask_edges
    ds[key].masks['center'] = mask_center

We can inspect that the coordinate corrections and masking were applied correctly by opening the instrument view.

In [ ]:
scn.instrument_view(ds['sample'], pixel_size=0.0075)

### Mask in time-of-flight

This could be implemented as masking specific time bins for a specific region in space,
but for now we keep it simple.

In [ ]:
tof = data.coords['tof']
tof_masked_region = sc.less(tof['tof',1:], 1500.0 * sc.units.us) | \
                         (sc.greater(tof['tof',:-1], 17500.0 * sc.units.us) & \
                          sc.less(tof['tof',1:], 19000.0 * sc.units.us))

for key in ds:
    ds[key].masks['bins'] = tof_masked_region
    

mask_tof_min = sc.scalar(13000.0, unit='us')
mask_tof_max = sc.scalar(15750.0, unit='us')
tof_masked_region = sc.concat([ds.coords['tof']['tof', 0],
                               mask_tof_min, mask_tof_max,
                               ds.coords['tof']['tof', -1]], dim='tof')

binned = sc.Dataset()
for key in ds:
    binned[key] = sc.bin(ds[key], edges=[tof_masked_region])
    binned[key].masks['bragg_peaks'] = sc.array(dims=['tof'], values=[False, True, False])
#binned

## Use to_I_of_Q workflow

We now reduce the sample and the background measurements to `Q` using the `sans.to_I_of_Q` workflow.

In that process,
the intensity as a function of `Q` is normalized using the direct measurement and direct beam function.

The workflow needs monitor data from the sample, background, and direct runs to compute the normalization.
It accepts those in the form of a dictionaries:

In [ ]:
#TODO: It needs to be sorted out
sample_monitors = {'incident': binned['sample'].attrs["monitor1"].value,
                   'transmission': binned['sample'].attrs["monitor2"].value}

direct_monitors = {'incident': binned['direct'].attrs["monitor1"].value,
                   'transmission': binned['direct'].attrs["monitor2"].value}

background_monitors = {'incident': binned['background'].attrs["monitor1"].value,
                       'transmission': binned['background'].attrs["monitor2"].value}

We then call the workflow on the sample and direct runs:

In [ ]:
sample_q = sans.to_I_of_Q(data=binned['sample'],
    data_monitors=sample_monitors,
    direct_monitors=direct_monitors,
    direct_beam=direct_beam,
    wavelength_bins=wavelength_bins,
    q_bins=q_bins,
    gravity=gravity,
    monitor_non_background_range=monitor_non_background_range)
sample_q.plot()

In [ ]:
background_q = sans.to_I_of_Q(data=binned['background'],
    data_monitors=background_monitors,
    direct_monitors=direct_monitors,
    direct_beam=direct_beam,
    wavelength_bins=wavelength_bins,
    q_bins=q_bins,
    gravity=gravity,
    monitor_non_background_range=monitor_non_background_range)
background_q.plot()

We are now in a position to subtract the background from the sample measurement:

In [ ]:
result = sample_q.bins.sum() - background_q.bins.sum()
result

In [ ]:
fig1, ax1 = plt.subplots(1, 2, figsize=(10, 4))
sc.plot(result, ax=ax1[0])
sc.plot(result, norm='log', ax=ax1[1])
fig1

<div class="alert alert-info">

**Note**

Instead of `.bins.sum()`,
one could use `sc.histogram()` above to define different `Q` bins compared to the ones defined at the top of the notebook.
This can be done in event mode, see [here](https://scipp.github.io/user-guide/binned-data/computation.html#Subtraction).

There may be performance advantages to first use a coarse `Q` binning when the computing `I(Q)` numerator,
and use finer binning for the final results.

</div>

## Wavelength bands

It is often useful to process the data in a small number (~10) of separate wavelength bands.

This can be achieved by requesting 10 bands from the `to_I_of_Q` workflow via the `wavelength_bands` argument.

In [ ]:
wavelength_bands = sc.linspace(dim='wavelength', start=2.0, stop=16.0, num=11,
                               unit='angstrom')

sample_slices = sans.to_I_of_Q(data=binned['sample'],
    data_monitors=sample_monitors,
    direct_monitors=direct_monitors,
    direct_beam=direct_beam,
    wavelength_bins=wavelength_bins,
    q_bins=q_bins,
    gravity=gravity,
    wavelength_bands=wavelength_bands,
    monitor_non_background_range=monitor_non_background_range)

background_slices = sans.to_I_of_Q(data=binned['background'],
    data_monitors=background_monitors,
    direct_monitors=direct_monitors,
    direct_beam=direct_beam,
    wavelength_bins=wavelength_bins,
    q_bins=q_bins,
    gravity=gravity,
    wavelength_bands=wavelength_bands,
    monitor_non_background_range=monitor_non_background_range)

result_slices = sample_slices.bins.sum() - background_slices.bins.sum()
result_slices

In [ ]:
collapsed = sc.collapse(result_slices, keep='Q')

fig2, ax2 = plt.subplots(1, 2, figsize=(10, 4))
sc.plot(collapsed, ax=ax2[0])
sc.plot(collapsed, norm='log', legend=False, ax=ax2[1])
fig2

## References

<div id="manasi2021"></div>

Manasi I., Andalibi M. R., Atri R. S., Hooton J., King S. M., Edler K. J., **2021**,
*Self-assembly of ionic and non-ionic surfactants in type IV cerium nitrate and urea based deep eutectic solvent*,
[J. Chem. Phys. 155, 084902](https://doi.org/10.1063/5.0059238)